In [ ]:
# --- load libraries ---
from semanticgis.abstract import Pipeline
from semanticgis.compilers import mermaid, qgis_recipy, validate
from IPython.display import display, Markdown, HTML, Image

# --- Define Pipeline ---
p = Pipeline(name="Pipeline Demo")

## Notebook Overview
This notebook demonstrates how to import `semanticgis.abstract.Pipeline`, register semantic inputs, and compile the resulting DAG into Mermaid and QGIS documentation. Always run Cell 1 first to ensure the library is imported before executing the remaining steps.

## 1.  Declare input and  Milestone Data -

In [2]:

p.io.declare_input(
    name="Europa_countries",
    source="https://raw.githubusercontent.com/Esbern/gis_demo_data/main/europa.geojson",
    spatial_nature="discrete",  # Defines it as a Vector/Object model
    data_model="vector",
    description="European sovereign boundaries",
    attributes={
        "SOVEREIGNT": {
            "scale": "nominal", 
            "description": "Full name of the sovereign state"
        },
        "POP_EST": {
            "scale": "ratio", 
            "description": "Estimated population count"
        }
    }
)

## 2. Declare the processing steps

In [3]:
# Isolate the Kingdom of Denmark
dk_handle = p.extraction.filter_by_sql(
    dataset="Europa_countries",
    where_clause="SOVEREIGNT = 'Denmark'",
    label="Kingdom of Denmark"
)

# Buffer to find the 12 NM maritime boundary
p.proximity.buffer(
    dataset=dk_handle,
    distance=0.22,  # approximately 12 nautical miles in degrees
    units="degrees",
    output_name="Danish territorial sea",
    label="12 NM territorial buffer"
)

# Buffer to find the 59 NM maritime boundary
p.proximity.buffer(
    dataset=dk_handle,
    distance=0.59,  # approximately 59 nautical miles in degrees
    units="degrees",
    output_name="Danish intrest zone sea",
    label="59 NM intrest zome"
)

## 3. Visualize and Validate the Plan --

In [4]:
display(Markdown(mermaid.compile(p)))

```mermaid
flowchart TD

    %% -- Nodes --
    node_1_data((<b>Europa_countries</b><br/>Declare Europa_countries<br/><small>vector / ratio / discrete</small>))
    node_2_data((Kingdom of Denmark<br/><small>vector / ratio / discrete</small>))
    node_3_data((<b>Danish territorial sea</b><br/>12 NM territorial buffer<br/><small>vector / ratio / discrete</small>))
    node_4_data((<b>Danish intrest zone sea</b><br/>59 NM intrest zome<br/><small>vector / ratio / discrete</small>))
    node_2_proc[Kingdom of Denmark<br/><small>extraction.filter_by_sql</small>]
    node_3_proc[12 NM territorial buffer<br/><small>proximity.buffer</small>]
    node_4_proc[59 NM intrest zome<br/><small>proximity.buffer</small>]

    %% -- Connections --
    node_1_data --> node_2_proc
    node_2_data --> node_3_proc
    node_2_data --> node_4_proc
    node_2_proc --> node_2_data
    node_3_proc --> node_3_data
    node_4_proc --> node_4_data
```

In [ ]:
# Render via mermaid.ink (graph) and keep the Markdown block for docs export.
try:
    graph_png = mermaid.compile(p, type="graph", dpi=175, return_bytes=True)
except RuntimeError as exc:
    display(Markdown(f"Mermaid render failed: {exc}"))
else:
    display(Image(data=graph_png))

```mermaid
flowchart TD

    %% -- Nodes --
    node_1_data((<b>Europa_countries</b><br/>Declare Europa_countries<br/><small>vector / ratio / discrete</small>))
    node_2_data((Kingdom of Denmark<br/><small>vector / ratio / discrete</small>))
    node_3_data((<b>Danish territorial sea</b><br/>12 NM territorial buffer<br/><small>vector / ratio / discrete</small>))
    node_4_data((<b>Danish intrest zone sea</b><br/>59 NM intrest zome<br/><small>vector / ratio / discrete</small>))
    node_2_proc[Kingdom of Denmark<br/><small>extraction.filter_by_sql</small>]
    node_3_proc[12 NM territorial buffer<br/><small>proximity.buffer</small>]
    node_4_proc[59 NM intrest zome<br/><small>proximity.buffer</small>]

    %% -- Connections --
    node_1_data --> node_2_proc
    node_2_data --> node_3_proc
    node_2_data --> node_4_proc
    node_2_proc --> node_2_data
    node_3_proc --> node_3_data
    node_4_proc --> node_4_data
```

## 4. Display a QGIS recipy

In [6]:
display(Markdown(qgis_recipy.compile(p)))

# QGIS Recipe: Pipeline Demo

---

### Step 1: Declare Europa_countries

**Action**: Load a vector layer and register it as `Europa_countries`.

* **Details**: Use `Layer > Add Layer > Add Vector Layer...` and browse to `https://raw.githubusercontent.com/Esbern/gis_demo_data/main/europa.geojson`. Record metadata such as CRS `None` if provided.

---

### Step 2: Kingdom of Denmark

**Action**: Run a SQL query using the DB Manager or Virtual Layer feature.

* **Details**:
    * *Source layer*: `Declare Europa_countries`
    * *WHERE clause*: `SOVEREIGNT = 'Denmark'`

---

### Step 3: 12 NM territorial buffer

**Action**: Run the Processing Toolbox → Buffer tool.

* **Settings**:
    * *Input*: `Kingdom of Denmark`
    * *Distance*: `0.22 degrees`
    * Ensure the layer is in an appropriate projected CRS before buffering.

---

### Step 4: 59 NM intrest zome

**Action**: Run the Processing Toolbox → Buffer tool.

* **Settings**:
    * *Input*: `Kingdom of Denmark`
    * *Distance*: `0.59 degrees`
    * Ensure the layer is in an appropriate projected CRS before buffering.

## 5. Validate the pipeline semantics

In [7]:
validate.compile(p)

✅ SUCCESS: All workflow checks passed!


## 6. Execution placeholder
The Python executor is paused during the functional-complex refactor. Once it returns, this section will demonstrate how to materialise the territorial sea features and render them in Folium.